## Setup

In [1]:
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf

from pyMLaux import show_img_data, plot_history, evaluate_regression_result
 
# Read data
base_dir = r"C:\Users\kilia\MASTER\rlpharm\data"
data_path = base_dir + "/approxFinal.csv"
model_save_path = base_dir + "/models/approximator/best.pt"

df = pd.read_csv(data_path)
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values.reshape(-1, 1)
 
# train-test split for model evaluation
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.7, shuffle=True)

def create_hyperparams(n, epochs, bounds, hidden_layer_sizes, dropout, actimel, default_lr=0.001):
    df = pd.DataFrame(index=range(n),
                      columns=['no_hidden_layers', 'hidden_layers', 'activation', 'dropout', 'lr', 'epochs'])

    for i in range(n):
        df.loc[i, 'lr'] = default_lr * 5.**random.uniform(-1., 0.5)
        df.loc[i, 'epochs'] = random.sample(epochs, 1)[0]
    
        no_layers = random.randint(bounds[0],bounds[1])
        df.loc[i, 'no_hidden_layers'] = no_layers 
        df.loc[i, 'hidden_layers'] = [int(random.sample(hidden_layer_sizes, 1)[0]) for i in range(no_layers)]
        df.loc[i, 'dropout'] = random.sample(dropout, 1)[0]
        df.loc[i, 'activation'] = random.sample(actimel, 1)[0] # für zusätzliche abwehrkräfte gegen schlechte models
    
    return(df)

def create_network(hp, no_inputs=7, no_outputs=1, output_activation='softmax', **kwargs):
    hidden_layers = hp['hidden_layers']
    
    dropout = hp['dropout']
    hidden_activation = hp['activation']
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(no_inputs, )))

    for cl in hidden_layers:
        model.add(tf.keras.layers.Dense(cl, activation=hidden_activation))
        if dropout > 0:
            model.add(tf.keras.layers.Dropout(dropout))
            
    model.add(tf.keras.layers.Dense(no_outputs, activation=output_activation)) 

    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['mse', 'mae'])

    return(model)

def find_best(df, crit='MAE'):
    index = np.where(df[crit] == np.amax(df[crit]))[0]
    return(df.iloc[list(index), :])



## Parameters

In [2]:
batch_size = 32
no_models = 50
model_sel = create_hyperparams(50, [50, 60, 80, 90], (3,4), [300,400,500], [0.5, 0.4, 0.3], ['relu'], 0.001)
model_sel['MAE'] = -1
for i in tqdm(range(no_models)):
    model = create_network(model_sel.iloc[i])

    history = model.fit(x=X_train, y=y_train, 
                        epochs=model_sel['epochs'][i],
                        batch_size=batch_size, 
                        verbose=0)

    pred = model.predict(x=X_val)
    predC = np.argmax(pred, axis=1)

    model_sel.loc[i, 'MAE'] = mean_absolute_error(y_val, predC)

    tf.keras.backend.clear_session()

model_sel.sort_values(by='MAE', ascending=False).head(10)

  0%|          | 0/50 [00:00<?, ?it/s]

34/34 [==============================] - 0s 788us/step


,no_hidden_layers,hidden_layers,activation,dropout,lr,epochs,MAE
0,3,"[400, 400, 300]",relu,0.3,0.000262,80,4.66558
37,4,"[400, 400, 500, 400]",relu,0.4,0.000927,50,4.66558
27,3,"[300, 400, 300]",relu,0.3,0.001442,50,4.66558
28,4,"[500, 400, 400, 300]",relu,0.3,0.000364,50,4.66558
29,3,"[300, 500, 500]",relu,0.4,0.002068,90,4.66558
30,4,"[300, 500, 300, 400]",relu,0.3,0.000498,80,4.66558
31,4,"[400, 300, 300, 500]",relu,0.4,0.000287,80,4.66558
32,3,"[300, 400, 500]",relu,0.5,0.000455,50,4.66558
33,3,"[400, 500, 300]",relu,0.3,0.000468,90,4.66558
34,3,"[400, 400, 400]",relu,0.4,0.002015,90,4.66558
